In [56]:
import scipy.io as sc
import numpy as np
from sklearn.svm import LinearSVC,SVC
import copy
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from HelperFunctions_ import *
import tables as tab
from SVMBDT import *
import struct 

In [7]:
#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

#trondheim_new_labels = copy.deepcopy(new_trondheim_gt)

#trondheim_new_labels[trondheim_new_labels == 10] = 9
#trondheim_new_labels[trondheim_new_labels == 9] = 2

#nl = [4,5,6,7,8]

#for i in range(len(nl)):
#        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

NameError: name 'new_trondheim_gt' is not defined

In [8]:
trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

In [9]:
#Preprocess the labeling 

new_trondheim_gt = []
y = trondheim_gt.flatten()

for i in range(0,len(y),3): 
    new_trondheim_gt.append(y[i])

new_trondheim_gt = np.array(new_trondheim_gt)

In [10]:
def rotate_image(image): 
    rotated_image = []
    for i in range(len(image[0])): 
        x_array = []
        for j in range(len(image)-1,-1,-1):
            x_array.append(image[j][i])
        rotated_image.append(x_array)

    return np.array(rotated_image)

In [39]:
#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)


#trondheim_new_labels[trondheim_new_labels == 10] = 9
#trondheim_new_labels[trondheim_new_labels == 9] = 3

nl = [4,5,6,7,8]

for i in range(len(nl)):
        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

classesInLabels(trondheim_new_labels)

array([ 2,  3,  9, 10], dtype=int16)

In [40]:
new_trondheim = rotate_image(trondheim_data)

In [54]:
#Finding path names 
Mjosa_data = split_dataset_paths("../../2022_07_11_Mjosa/2022_07_11_Mjosa/", ["he5", "csv", "jpg"])

In [58]:
#Amount of dataset should be handled
N_datasets = 4

#Open files 
open_files_array = []
for i in range(N_datasets):
    file = tab.open_file(Mjosa_data["he5"][i])
    open_files_array.append(file.get_node("/HDFEOS/SWATHS/PRS_L1_HCO/")['Data Fields']['VNIR_Cube'][:,5:,:])

#Load the labels 
load_labels_array = []
for i in range(N_datasets):
    file = Mjosa_data["csv"][i]
    load_labels_array.append(np.loadtxt(file, dtype=np.ubyte))

In [59]:
data = open_files_array[1]
labels = load_labels_array[1]

In [102]:
# 1) Spruce, 2) Pine , 3) Decidous (leaf), 4) Water, 5) Clouds, and 0) other
labels[labels == 3] = 1 
labels[labels == 2] = 1 

In [86]:
f1 = tab.open_file("../../2022_07_11_Mjosa/2022_07_11_Mjosa/PRS_L1_STD_OFFL_20220711104640_20220711104644_0001.he5")

d3 = f1.get_node("/HDFEOS/SWATHS/PRS_L1_HRC/")

mjosa_new_cube = np.zeros((1000,1000,3))
mjosa_new_cube[:,:,0] = d3['Data Fields']['VNIR_Cube'][:,31,:]
mjosa_new_cube[:,:,1] = d3['Data Fields']['VNIR_Cube'][:,43,:]
mjosa_new_cube[:,:,2] = d3['Data Fields']['VNIR_Cube'][:,56,:]
mjosa_new_cube /= mjosa_new_cube.max()/2
mjosa_new_cube[mjosa_new_cube > 1 ] = 1

In [ ]:

plt.imshow(np.array(mjosa_new_cube))
plt.axis('off')
#plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
plt.savefig("Mjosa_data", dpi = 300, bbox_inches="tight")
plt.close()

In [103]:
X,y = preprocessing_data(mjosa_new_cube,labels,3)

In [108]:
classesInLabels(y)

array([0, 1, 4, 5], dtype=uint8)

In [105]:
#X = np.array(new_trondheim).reshape((-1,3))
#X = preprocessing.scale(X, axis=0)
#y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

In [106]:
tb = {0 : [[3,9,10],[2]], 1 : [[[3],[9,10]],[]], 2 : [[[9],[10]],[]]}
svm_tb = {0 : LinearSVC(), 1 : [LinearSVC(), False] , 2 : [LinearSVC(), False]}
svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

In [109]:
# 1) Spruce, 2) Pine , 3) Decidous (leaf), 4) Water, 5) Clouds, and 0) other
#tb = {0 : [[4],[0,1,2,3,5]], 1 : [[],[[5],[0,1,2,3]]], 2 : [[],[[0],[1,2,3]]], 3 : [[],[[1],[2,3]]], 4 : [[],[[2],[3]]]}

iter = 1e5

"""
sbm = {0 : LinearSVC(class_weight= "balanced", max_iter=iter),
       1 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)], 
       2 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)],
       3 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)], 
       4 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)]}
"""
tb = {0 : [[4],[0,1,5]], 1 : [[],[[0],[1,5]]], 2 : [[],[[1],[5]]]}

sbm = {0 : LinearSVC(class_weight= "balanced", max_iter=iter),
       1 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)], 
       2 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)]}


svm_tree_branches = SvmDesionTreeTrain(X_train, y_train, tb, sbm)

The training time is: 28.414 sec.


In [110]:
test_name = "vigo_2023-02-09_1106Z-bin3.png"
test_data = plt.imread("../Training_data/" + test_name)

In [111]:
X_test = np.array(test_data).reshape((-1,3))
X_test = preprocessing.scale(X_test, axis=0)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [112]:
pl = SvmDesionTreePredict(X_test, svm_tree_branches, tb, sub_data = 0)

The first branch: 0.005
The prediction time is: 0.599 sec.


In [113]:
plt.imshow(pl.reshape(len(test_data),len(test_data[0])))
plt.axis('off')
plt.savefig("../Z_Labeled_HYSPO_Data/" + test_name.split(".")[0] + "labeled", dpi = 300, bbox_inches="tight")
plt.close()

In [4]:
0 : 01
1 : 02
2 : 03
3 : 04
...
15 : 016 
16 : 10
17 : 11
18 : 12 





0


In [76]:
num = 17
(num%16) 

1

In [ ]:
def int_to_svm_name(num): 
    
    first_num = 
    second_num = num%16 

    return 0 

In [129]:
def float_to_int_delta(value, byte): 
    delta = 2/(pow(2,8*byte)-1)
    return int(value/delta) 

def int_to_float_delta(value, byte): 
    delta = 2/(pow(2,8*byte)-1)
    return value*delta




0.6699931334401464


In [ ]:
def training_data_to_binary_file_2_byte(file_path ,modus, class_1, class_2, b, w_i):
    #svm_model format: [class1, class2, b, w_i[]]

    #Modus: type of svm model. Linear, rbf, sparse etc.

    svm_model_name = ""

    if(modus == 0): 
        svm_model_name = "lsm" + str(class_1) + str(class_2)

    file = open(file_path + svm_model_name + ".bin", "wb")    
    
    buf = struct.pack("BB", class_1, class_2)

    file.write(buf)

    buf = struct.pack(i'{len([b])}i', *[b])

    file.write(buf)

    buf = struct.pack(i'{len(w_i)}i', *w_i)

    file.write(buf)

    file.close()